# TO DO
 * Add exogenous variables

In [4]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
import numpy as np

# Hyperparameters

In [ ]:
num_samples = 500
time_steps = 10
features = 5
batch_size = 1
epochs = 10

# Generate Data
Generated data must be of the shape (num_samples, time_steps, features).

In [13]:
# Generate a time series
X = np.random.normal(0,1,(num_samples, features))

# Use Keras to create a generator with both time series sequences and their targets.
data_gen = TimeseriesGenerator(X, X,
                               length=time_steps, sampling_rate=1,
                               batch_size=batch_size)

# Multivariate Time Series Prediction with Keras LSTM

In [14]:
model = Sequential()
model.add(LSTM(units=30, return_sequences= True, input_shape=(time_steps,features)))
model.add(LSTM(units=30, return_sequences=True))
model.add(LSTM(units=30))
model.add(Dense(units=features))

model.summary()

model.compile(optimizer='adam', loss='mean_squared_error')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_9 (LSTM)                (None, 10, 30)            4320      
_________________________________________________________________
lstm_10 (LSTM)               (None, 10, 30)            7320      
_________________________________________________________________
lstm_11 (LSTM)               (None, 30)                7320      
_________________________________________________________________
dense_5 (Dense)              (None, 5)                 155       
Total params: 19,115
Trainable params: 19,115
Non-trainable params: 0
_________________________________________________________________


In [15]:
#model.fit(X, Y, epochs=20, batch_size=32)

model.fit_generator(data_gen, steps_per_epoch = len(data_gen), epochs = epochs)

Epoch 1/10
490/490 [==============================] - 13s 26ms/step - loss: 0.9813
Epoch 2/10
490/490 [==============================] - 11s 22ms/step - loss: 0.9788
Epoch 3/10
490/490 [==============================] - 11s 22ms/step - loss: 0.9772
Epoch 4/10
490/490 [==============================] - 11s 22ms/step - loss: 0.9746
Epoch 5/10
490/490 [==============================] - 11s 22ms/step - loss: 0.9722
Epoch 6/10
490/490 [==============================] - 10s 21ms/step - loss: 0.9677
Epoch 7/10
490/490 [==============================] - 11s 21ms/step - loss: 0.9640
Epoch 8/10
490/490 [==============================] - 10s 21ms/step - loss: 0.9551
Epoch 9/10
490/490 [==============================] - 10s 21ms/step - loss: 0.9519
Epoch 10/10
490/490 [==============================] - 11s 21ms/step - loss: 0.9417


# 1-Dimensional CNN for Multivariate Time Series Prediction

In [18]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D

In [20]:
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(time_steps,features)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(features))
model.compile(optimizer='adam', loss='mse')

In [21]:
model.fit_generator(data_gen, steps_per_epoch = len(data_gen), epochs = epochs)

Epoch 1/10
490/490 [==============================] - 2s 4ms/step - loss: 1.0222
Epoch 2/10
490/490 [==============================] - 1s 2ms/step - loss: 0.9684
Epoch 3/10
490/490 [==============================] - 1s 2ms/step - loss: 0.9292
Epoch 4/10
490/490 [==============================] - 1s 2ms/step - loss: 0.8884
Epoch 5/10
490/490 [==============================] - 1s 2ms/step - loss: 0.8388
Epoch 6/10
490/490 [==============================] - 1s 2ms/step - loss: 0.7722
Epoch 7/10
490/490 [==============================] - 1s 2ms/step - loss: 0.7113
Epoch 8/10
490/490 [==============================] - 1s 2ms/step - loss: 0.6465
Epoch 9/10
490/490 [==============================] - 1s 2ms/step - loss: 0.5831
Epoch 10/10
490/490 [==============================] - 1s 2ms/step - loss: 0.5284


# 1-Dimensional LSTM-CNN

 * From <a href="https://www.kaggle.com/dimitreoliveira/deep-learning-for-time-series-forecasting">this post</a>.
 
 Input shape [samples, subsequences, timesteps, features]
 
From <a href="https://machinelearningmastery.com/how-to-get-started-with-deep-learning-for-time-series-forecasting-7-day-mini-course/">here</a>:
 
"The benefit of this model is that the model can support very long input sequences that can be read as blocks or subsequences by the CNN model, then pieced together by the LSTM model.

We can define a simple univariate problem as a sequence of integers, fit the model on this sequence and have the model predict the next value in the sequence. We will frame the problem to have 4 inputs and 1 output, for example: [10, 20, 30, 40] as input and [50] as output.

When using a hybrid CNN-LSTM model, we will further divide each sample into further subsequences. The CNN model will interpret each sub-sequence and the LSTM will piece together the interpretations from the subsequences. As such, we will split each sample into 2 subsequences of 2 times per subsequence.

The CNN will be defined to expect 2 time steps per subsequence with one feature. The entire CNN model is then wrapped in TimeDistributed wrapper layers so that it can be applied to each subsequence in the sample. The results are then interpreted by the LSTM layer before the model outputs a prediction."

In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import RepeatVector
from tensorflow.keras.layers import TimeDistributed

In [30]:
n_outputs = 1

In [33]:
model_cnn_lstm = Sequential()

model_cnn_lstm.add(TimeDistributed(
                                   Conv1D(filters = 64, kernel_size = 1, activation = 'relu'), 
                                   input_shape = (None, time_steps, features)
                                  )
                  )

model_cnn_lstm.add(TimeDistributed(MaxPooling1D(pool_size = 2)))
model_cnn_lstm.add(TimeDistributed(Flatten()))
model_cnn_lstm.add(LSTM(50, activation = 'relu'))
model_cnn_lstm.add(Dense(features))
model_cnn_lstm.compile(loss = 'mse', optimizer = 'adam')

In [34]:
model_cnn_lstm.fit_generator(data_gen, steps_per_epoch = len(data_gen), epochs = epochs)

Epoch 1/10


ValueError: Error when checking input: expected time_distributed_7_input to have 4 dimensions, but got array with shape (1, 10, 5)

# 2-D LSTM-CNN
 * From <a href="https://arxiv.org/abs/1506.04214v1">Convolutional LSTM Network: A Machine Learning Approach for Precipitation Nowcasting</a>.